In [20]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing import image_dataset_from_directory
from keras.applications import MobileNetV3Small
from keras.applications.mobilenet_v3 import preprocess_input


In [21]:
#Loading data

train_data = image_dataset_from_directory(
    directory="../data/SkinDisease/train",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(244,244),
    batch_size=32

)

test_data = image_dataset_from_directory(
    directory="../data/SkinDisease/test",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(244,244),
    batch_size=32,
    shuffle=False
)

Found 13898 files belonging to 22 classes.
Found 1546 files belonging to 22 classes.


In [22]:
# Preprocessing images
def preprocess(image, label):
    image = tf.image.resize(image, (224,224))
    image = preprocess_input(image)

    return image, label

train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)


In [23]:
train_data = train_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [24]:
augment_data = Sequential([
    
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomFlip('vertical'),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomRotation(0.05)

])

In [25]:
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [26]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

In [27]:
print(len(base_model.layers))

157


In [28]:
# Building model
model = Sequential()

model.add(keras.layers.Input(shape=(224,224,3)))
model.add(augment_data)
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.12))

model.add(Dense(22, activation='softmax'))

In [29]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 22)             │         2,838 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,015,814 (3.88 MB)

 Trainable params: 664,886 (2.54 MB)

 Non-trainable params: 350,928 (1.34 MB)

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=['accuracy'])

In [32]:
history = model.fit(train_data, epochs=20, validation_data=test_data, batch_size=64)

Epoch 1/20


435/435 ━━━━━━━━━━━━━━━━━━━━ 314s 672ms/step - accuracy: 0.0865 - loss: 3.0734 - val_accuracy: 0.1345 - val_loss: 2.9089
Epoch 2/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 214s 493ms/step - accuracy: 0.1728 - loss: 2.8204 - val_accuracy: 0.2070 - val_loss: 2.7101
Epoch 3/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 219s 504ms/step - accuracy: 0.2164 - loss: 2.6643 - val_accuracy: 0.2529 - val_loss: 2.5648
Epoch 4/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 215s 494ms/step - accuracy: 0.2487 - loss: 2.5572 - val_accuracy: 0.2768 - val_loss: 2.4516
Epoch 5/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 212s 487ms/step - accuracy: 0.2744 - loss: 2.4728 - val_accuracy: 0.2943 - val_loss: 2.3633
Epoch 6/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 211s 485ms/step - accuracy: 0.2946 - loss: 2.3982 - val_accuracy: 0.3131 - val_loss: 2.2889
Epoch 7/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 211s 485ms/step - accuracy: 0.3103 - loss: 2.3433 - val_accuracy: 0.3351 - val_loss: 2.2240
Epoch 8/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 211s 485ms/step - accuracy: 0.3296 - loss: 2.28